In [ ]:
# Retrieve all URLs from a Microsoft Learn Module

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
# DP-300
list_of_learn_paths = [
    "https://learn.microsoft.com/en-us/training/paths/introduction-to-azure-database-administration/",
    "https://learn.microsoft.com/en-us/training/paths/plan-implement-data-platform-resources/",
    "https://learn.microsoft.com/en-us/training/paths/implement-secure-environment-database-service/",
    "https://learn.microsoft.com/en-us/training/paths/monitor-optimize-operational-resources-sql-server/",
    "https://learn.microsoft.com/en-us/training/paths/optimize-query-performance-sql-server/",
    "https://learn.microsoft.com/en-us/training/paths/automate-tasks-sql-server/",
    "https://learn.microsoft.com/en-us/training/paths/plan-implement-high-availability-disaster-recovery-environment/"
]

In [ ]:
# DP-600
list_of_learn_paths = [
    "https://learn.microsoft.com/en-us/training/paths/ingest-data-with-microsoft-fabric/",
    "https://learn.microsoft.com/en-us/training/paths/implement-lakehouse-microsoft-fabric/",
    "https://learn.microsoft.com/en-us/training/paths/work-with-data-warehouses-using-microsoft-fabric/",
    "https://learn.microsoft.com/en-us/training/paths/work-semantic-models-microsoft-fabric/",
    "https://learn.microsoft.com/en-us/training/paths/design-build-tabular-models/",
    "https://learn.microsoft.com/en-us/training/paths/manage-analytics-development-lifecycle/"
]

In [ ]:
data = []

for url in list_of_learn_paths:
    learn_path = requests.get(url)
    soup_learnpath = BeautifulSoup(learn_path.text, "html.parser")
    links_learnpath = soup_learnpath.find_all(class_="font-size-h6")
    title_learnpath = soup_learnpath.find("h1", class_="title").text
    absolute_urls = [urljoin(url, link["href"]) for link in links_learnpath]

    print(title_learnpath)

    jsondata_learnpath = {} 
    jsondata_learnpath["learning_path"] = title_learnpath
    jsondata_learnpath["url"] = url 
    jsondata_learnpath["learning_modules"] = []

    for module in absolute_urls:
        learn_module = requests.get(module)
        soup_learnmodule = BeautifulSoup(learn_module.text, "html.parser")
        links_units = soup_learnmodule.find_all(class_="unit-title")
        links_units = [link for link in links_units if not any(keyword in link["href"] for keyword in ["exercise", "knowledge-check", "summary"])]
        title_module = soup_learnmodule.find("h1", class_="title").text
        absolute_urls_units = [urljoin(module, link["href"]) for link in links_units]

        print("- " + title_module)

        jsondata_learnmodule = {} 
        jsondata_learnmodule["learning_module"] = title_module
        jsondata_learnmodule["url"] = module 
        jsondata_learnmodule["learning_units"] = absolute_urls_units

        jsondata_learnpath["learning_modules"].append(jsondata_learnmodule)

    data.append(jsondata_learnpath)
    
# Write the data to the JSON file
with open("LearningPaths.json", "w") as json_file:
    json.dump(data, json_file, indent=2)

print("Done!")
